In [2]:
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)
#ignore all Depricated Warning
simplefilter(action='ignore', category=DeprecationWarning)
simplefilter(action='ignore', category=UserWarning)
simplefilter(action='ignore', category=RuntimeWarning)
import pandas as pd
import io
import numpy as np
import os
import matplotlib.pyplot as plt

In [7]:
dum_train = pd.read_csv("F:/Project/application_train.csv")

In [4]:
dum_test = pd.read_csv("F:/Project/application_test.csv")

In [5]:
print('Training data shape: ', dum_train.shape)

Training data shape:  (307511, 122)


In [9]:
dum_train = pd.get_dummies(dum_train, drop_first=True)
dum_train=dum_train.dropna(axis=1)

In [13]:
dum_test= pd.get_dummies(dum_test, drop_first=True)
dum_test=dum_test.dropna(axis=1)

In [14]:
train_labels = dum_train['TARGET']
train_sk_id_curr = dum_train['SK_ID_CURR']
test_sk_id_curr = dum_test['SK_ID_CURR']

dum_train.drop('SK_ID_CURR', inplace=True, axis=1)
dum_test.drop('SK_ID_CURR', inplace=True, axis=1)

dum_train, dum_test = dum_train.align(dum_test, join = 'inner', axis = 1)
print('Training Features shape: ', dum_train.shape)
print('Testing Features shape: ', dum_test.shape)

Training Features shape:  (307511, 164)
Testing Features shape:  (48744, 164)


In [15]:
from sklearn.metrics import accuracy_score
import lightgbm as lgb

In [17]:
from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(dum_train, train_labels, test_size = 0.30, random_state = 2019,
                                                   stratify=train_labels)


In [19]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [21]:
d_train = lgb.Dataset(x_train, label=y_train)

param = {'num_leaves':150, 'objective':'binary','max_depth':7,'learning_rate':.05,'max_bin':200}
param['metric'] = ['auc', 'binary_logloss']

In [23]:
from datetime import datetime 
num_round=100

lgbm=lgb.train(param,d_train,num_round)

print("Completed")

Completed


In [29]:
ypred2=lgbm.predict(x_test)
ypred2[0:5]  # showing first 5 predictions

array([0.04845195, 0.08881859, 0.08238342, 0.04352715, 0.07851827])

In [30]:
for i in range(0,9769):
    if ypred2[i]>=.5:    
        ypred2[i]=1
    else:  
        ypred2[i]=0

In [31]:
accuracy_lgbm = accuracy_score(ypred2.round(),y_test)
accuracy_lgbm
y_test.value_counts()

0    84806
1     7448
Name: TARGET, dtype: int64

In [42]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, ypred2.round())

In [43]:
from sklearn.metrics import roc_auc_score
print('ROC AUC' ,roc_auc_score(y_test,ypred2))

ROC AUC 0.6418595185769667


cm

In [53]:
cm

array([[84773,    33],
       [ 7443,     5]], dtype=int64)

In [54]:
y_pred

array([0.        , 0.        , 0.        , ..., 0.08028352, 0.08925394,
       0.09266538])

In [89]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(ypred2.round(),y_test)

In [90]:
accuracy

0.9192663732737876

In [91]:
predictions = lgbm.predict(dum_test)

In [92]:
predictions.round()

array([0., 0., 0., ..., 0., 0., 0.])

In [93]:
submission = pd.DataFrame({'SK_ID_CURR': test_sk_id_curr, 'TARGET': predictions.round()})


In [94]:
submission.to_csv("F:/Project/submit_lgbm.csv", index = False)